In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 🔬 Skin Cancer Detection System - Complete Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thedatadudech/skin-cancer-detection/blob/main/skincancer_detector.ipynb)
[![GitHub](https://img.shields.io/badge/GitHub-Repository-blue?logo=github)](https://github.com/thedatadudech/skin-cancer-detection)

This comprehensive notebook demonstrates the complete development process of an advanced skin cancer detection system using deep learning with PyTorch. 

## 📋 What You'll Learn
- **Data Analysis**: Explore the HAM10000 medical imaging dataset
- **Model Development**: Compare EfficientNet, ResNet, and custom CNN architectures
- **Training Pipeline**: Implement robust training with data augmentation
- **Performance Evaluation**: Analyze model performance across 7 skin lesion types
- **Deployment Ready**: Export models for production use

## 🎯 Medical Context
Early detection of skin cancer, particularly melanoma, is crucial for successful treatment. This system assists healthcare professionals in preliminary screening by analyzing dermoscopic images.

**⚠️ Medical Disclaimer**: This tool is for educational and research purposes only. Always consult qualified healthcare professionals for medical diagnosis.

## 🚀 Quick Start
1. Click the "Open in Colab" button above
2. Run the setup cells to install dependencies
3. Download the HAM10000 dataset (instructions below)
4. Execute the training pipeline

---

## 🔧 Google Colab Setup

If running on Google Colab, execute the following cells to set up the environment:

In [ ]:
# Install dependencies for Google Colab
import sys

if 'google.colab' in sys.modules:
    print("🚀 Setting up Google Colab environment...")
    
    # Install required packages
    !pip install torch>=2.5.1 torchvision>=0.20.1 -q
    !pip install streamlit>=1.41.1 pandas>=2.2.3 numpy>=2.2.2 -q
    !pip install scikit-learn>=1.6.1 Pillow==10.0.0 tqdm>=4.67.1 -q
    !pip install matplotlib seaborn -q
    
    # Clone the repository
    !git clone https://github.com/thedatadudech/skin-cancer-detection.git
    %cd skin-cancer-detection
    
    print("✅ Environment setup complete!")
else:
    print("📝 Running in local environment")

In [ ]:
# Dataset setup for Google Colab
import os

if 'google.colab' in sys.modules:
    print("📊 Setting up dataset for Colab...")
    
    # Create data directories
    os.makedirs('data/images', exist_ok=True)
    
    print("""📥 Dataset Download Instructions:
    
    To use this notebook, you need to download the HAM10000 dataset:
    
    1. Visit: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/DBW86T
    2. Download:
       - HAM10000_images_part1.zip
       - HAM10000_images_part2.zip  
       - HAM10000_metadata.csv
    3. Upload to Colab and extract:
       - Extract zip files to data/images/
       - Place metadata.csv in data/
    
    Or use the sample dataset for demonstration purposes.""")
    
    # Option to create sample data for demonstration
    create_sample = input("Create sample data for demonstration? (y/n): ")
    if create_sample.lower() == 'y':
        # Create minimal sample dataset for demonstration
        import pandas as pd
        import numpy as np
        from PIL import Image
        
        # Create sample metadata
        sample_data = {
            'image_id': [f'sample_{i:03d}' for i in range(50)],
            'dx': np.random.choice(['nv', 'mel', 'bkl', 'bcc', 'akiec', 'vasc', 'df'], 50),
            'age': np.random.randint(20, 80, 50),
            'sex': np.random.choice(['male', 'female'], 50)
        }
        
        df_sample = pd.DataFrame(sample_data)
        df_sample.to_csv('data/HAM10000_metadata.csv', index=False)
        
        # Create sample images
        for img_id in sample_data['image_id']:
            # Create random RGB image
            img_array = np.random.randint(0, 256, (224, 224, 3), dtype=np.uint8)
            img = Image.fromarray(img_array)
            img.save(f'data/images/{img_id}.jpg')
        
        print("✅ Sample dataset created for demonstration")
else:
    print("📂 Using local dataset")

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import os
import sys
from pathlib import Path

# PyTorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torchvision import models

# ML utilities
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

# Set random seeds for reproducibility
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

# Configure matplotlib
plt.style.use('default')
sns.set_palette("husl")

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 1. Data Loading and Initial Exploration

In [ ]:
#Check if mounted to load other path
if os.path.exists("../drive"):
    print("Google Drive is mounted.")
    source_path = "../drive/MyDrive/SkinCancerDetector/"
else:
    print("Google Drive is not mounted.")
    source_path = "data/"

In [ ]:
# Load and explore metadata
try:
    df = pd.read_csv(os.path.join(source_path, 'HAM10000_metadata'))
    print(f"✅ Dataset loaded successfully!")
    print(f"Dataset shape: {df.shape}")
    print(f"\nColumns: {list(df.columns)}")
    
    # Display first few rows
    display(df.head())
    
except FileNotFoundError:
    print("❌ Dataset not found. Please follow the dataset setup instructions above.")
    df = None

In [ ]:
if df is not None:
    # Define class mappings
    class_names = {
        'akiec': 'Actinic keratoses',
        'bcc': 'Basal cell carcinoma', 
        'bkl': 'Benign keratosis',
        'df': 'Dermatofibroma',
        'mel': 'Melanoma',
        'nv': 'Melanocytic nevi',
        'vasc': 'Vascular lesions'
    }
    
    # Display class distribution
    print("📊 Class Distribution:")
    class_counts = df['dx'].value_counts()
    for code, count in class_counts.items():
        print(f"{class_names.get(code, code)}: {count} ({count/len(df)*100:.1f}%)")
    
    # Visualize class distribution
    plt.figure(figsize=(12, 6))
    ax = sns.countplot(data=df, x='dx', order=class_counts.index)
    plt.title('Distribution of Skin Lesion Types in HAM10000 Dataset', fontsize=14, fontweight='bold')
    plt.xlabel('Lesion Type')
    plt.ylabel('Number of Images')
    
    # Add count labels on bars
    for i, v in enumerate(class_counts.values):
        ax.text(i, v + 50, str(v), ha='center', va='bottom')
    
    # Update x-axis labels with full names
    ax.set_xticklabels([class_names.get(code, code) for code in class_counts.index], 
                       rotation=45, ha='right')
    
    plt.tight_layout()
    plt.show()
    
    # Additional statistics
    print(f"\n📈 Dataset Statistics:")
    print(f"Total images: {len(df):,}")
    print(f"Number of classes: {df['dx'].nunique()}")
    print(f"Age range: {df['age'].min():.0f} - {df['age'].max():.0f} years")
    print(f"Gender distribution: {df['sex'].value_counts().to_dict()}")

## 2. Image Analysis

In [ ]:
def analyze_image_properties(data_dir, sample_size=10):
    """Analyze properties of images in the dataset"""
    image_files = np.random.choice(os.listdir(data_dir), sample_size)
    widths, heights = [], []
    
    for img_file in image_files:
        img = Image.open(os.path.join(data_dir, img_file))
        widths.append(img.size[0])
        heights.append(img.size[1])
    
    return widths, heights

widths, heights = analyze_image_properties(os.path.join(source_path, 'images'))

plt.figure(figsize=(12, 4))
plt.subplot(121)
plt.hist(widths)
plt.title('Image Widths Distribution')
plt.subplot(122)
plt.hist(heights)
plt.title('Image Heights Distribution')
plt.show()

## 3. Data Preparation

In [ ]:
# Use project modules if available, otherwise implement inline
try:
    from src.data_loader import DataLoader as ProjectDataLoader
    from src.model import create_model
    from src.preprocessing import get_data_transforms
    print("✅ Using project modules")
    use_project_modules = True
except ImportError:
    print("📝 Project modules not available, using inline implementations")
    use_project_modules = False

if df is not None and use_project_modules:
    # Use the existing DataLoader from the project
    print("🔧 Setting up data loaders using project DataLoader...")
    
    # Initialize the project DataLoader
    data_loader = ProjectDataLoader(
        data_dir=os.path.join(source_path, 'images'),
        metadata_path=os.path.join(source_path, 'HAM10000_metadata'),
        batch_size=BATCH_SIZE
    )
    
    # Create data loaders
    train_loader, val_loader, test_loader = data_loader.create_data_loaders()
    
    # Load metadata for class mapping
    metadata_df = data_loader.load_metadata()
    class_names = list(metadata_df['dx'].unique())
    
    print(f"\n📊 Data loaders created successfully:")
    print(f"Training batches: {len(train_loader)}")
    print(f"Validation batches: {len(val_loader)}")
    print(f"Test batches: {len(test_loader)}")
    print(f"\n🏷️ Classes: {class_names}")
    
elif df is not None:
    print("⚠️ Project modules not available, manual data splitting needed")
    # Fallback implementation without project modules
    # Create label mapping
    label_mapping = {label: idx for idx, label in enumerate(df['dx'].unique())}
    print(f"🏷️ Label mapping: {label_mapping}")
    
    # Add numeric labels to dataframe
    df['label'] = df['dx'].map(label_mapping)
    
    # Split data
    train_df, temp_df = train_test_split(df, test_size=0.3, stratify=df['label'], random_state=42)
    val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)
    
    print(f"\n📊 Data splits:")
    print(f"Training: {len(train_df)} images")
    print(f"Validation: {len(val_df)} images")
    print(f"Test: {len(test_df)} images")
else:
    print("❌ No dataset available")

## 4. Model Development and Training

We'll implement and compare three different architectures:
- **EfficientNet-B0**: State-of-the-art efficient architecture
- **ResNet-50**: Popular residual network
- **Custom CNN**: Baseline convolutional network

In [ ]:
# Define model architectures
def create_efficientnet_model(num_classes=7):
    """Create EfficientNet-B0 model with transfer learning"""
    model = models.efficientnet_b0(pretrained=True)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    return model

def create_resnet_model(num_classes=7):
    """Create ResNet-50 model with transfer learning"""
    model = models.resnet50(pretrained=True)
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

class CustomCNN(nn.Module):
    """Custom CNN for comparison"""
    def __init__(self, num_classes=7):
        super(CustomCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
        )
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((7, 7)),
            nn.Flatten(),
            nn.Linear(128 * 7 * 7, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
    
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Model configurations
NUM_CLASSES = 7
BATCH_SIZE = 16  # Reduced for better Colab compatibility
EPOCHS = 5  # Reduced for demonstration
LEARNING_RATE = 1e-4

print(f"🔧 Training Configuration:")
print(f"Number of classes: {NUM_CLASSES}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Epochs: {EPOCHS}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Device: {device}")

In [ ]:
# Create model instances using project modules or inline implementations
if use_project_modules:
    # Use project model creation functions
    models = {
        'EfficientNet': create_model('efficientnet', num_classes=NUM_CLASSES),
        'ResNet': create_model('resnet', num_classes=NUM_CLASSES),
        'CustomCNN': create_model('custom_cnn', num_classes=NUM_CLASSES)
    }
    print("✅ Models created using project functions")
else:
    # Fallback inline model definitions for Colab
    def create_efficientnet_model(num_classes=7):
        model = models.efficientnet_b0(pretrained=True)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
        return model

    def create_resnet_model(num_classes=7):
        model = models.resnet50(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
        return model

    class CustomCNN(nn.Module):
        def __init__(self, num_classes=7):
            super(CustomCNN, self).__init__()
            self.features = nn.Sequential(
                nn.Conv2d(3, 32, 3, padding=1),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2),
                nn.Conv2d(32, 64, 3, padding=1),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2),
                nn.Conv2d(64, 128, 3, padding=1),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2),
            )
            self.classifier = nn.Sequential(
                nn.AdaptiveAvgPool2d((7, 7)),
                nn.Flatten(),
                nn.Linear(128 * 7 * 7, 512),
                nn.ReLU(inplace=True),
                nn.Dropout(0.5),
                nn.Linear(512, num_classes)
            )
        
        def forward(self, x):
            x = self.features(x)
            x = self.classifier(x)
            return x

    models = {
        'EfficientNet': create_efficientnet_model(NUM_CLASSES),
        'ResNet': create_resnet_model(NUM_CLASSES),
        'CustomCNN': CustomCNN(NUM_CLASSES)
    }
    print("✅ Models created using inline implementations")

# Display model information
print(f"\n🔧 Models created:")
for name, model in models.items():
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"{name}: {total_params:,} total parameters, {trainable_params:,} trainable")

In [ ]:
# Create model instances using project modules or inline implementations
if use_project_modules:
    # Use project model creation functions
    models = {
        'EfficientNet': create_model('efficientnet', num_classes=NUM_CLASSES),
        'ResNet': create_model('resnet', num_classes=NUM_CLASSES),
        'CustomCNN': create_model('custom_cnn', num_classes=NUM_CLASSES)
    }
    print("✅ Models created using project functions")
else:
    # Fallback inline model definitions for Colab
    def create_efficientnet_model(num_classes=7):
        model = models.efficientnet_b0(pretrained=True)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
        return model

    def create_resnet_model(num_classes=7):
        model = models.resnet50(pretrained=True)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
        return model

    class CustomCNN(nn.Module):
        def __init__(self, num_classes=7):
            super(CustomCNN, self).__init__()
            self.features = nn.Sequential(
                nn.Conv2d(3, 32, 3, padding=1),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2),
                nn.Conv2d(32, 64, 3, padding=1),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2),
                nn.Conv2d(64, 128, 3, padding=1),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(2),
            )
            self.classifier = nn.Sequential(
                nn.AdaptiveAvgPool2d((7, 7)),
                nn.Flatten(),
                nn.Linear(128 * 7 * 7, 512),
                nn.ReLU(inplace=True),
                nn.Dropout(0.5),
                nn.Linear(512, num_classes)
            )
        
        def forward(self, x):
            x = self.features(x)
            x = self.classifier(x)
            return x

    models = {
        'EfficientNet': create_efficientnet_model(NUM_CLASSES),
        'ResNet': create_resnet_model(NUM_CLASSES),
        'CustomCNN': CustomCNN(NUM_CLASSES)
    }
    print("✅ Models created using inline implementations")

# Display model information
print(f"\n🔧 Models created:")
for name, model in models.items():
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"{name}: {total_params:,} total parameters, {trainable_params:,} trainable")

## 5. Model Training and Evaluation

Jetzt trainieren wir unsere PyTorch-Modelle und vergleichen deren Performance.

In [ ]:
# Train models and store results
if use_project_modules and 'train_loader' in locals():
    training_histories = {}
    
    # Train each model
    for name, model in models.items():
        print(f"\n🚀 Training {name}...")
        print("=" * 50)
        
        try:
            history = train_pytorch_model(model, train_loader, val_loader, num_epochs=EPOCHS)
            training_histories[name] = history
            print(f"✅ {name} training completed successfully!")
        except Exception as e:
            print(f"❌ Error training {name}: {str(e)}")
            continue
        
        print("\n" + "="*50 + "\n")
        
elif not use_project_modules and 'train_df' in locals():
    print("⚠️ Manual training setup needed without project modules")
    print("This would require implementing custom DataLoader for the manual data splits")
    print("For best results, use the complete project environment")
    
else:
    print("❌ Cannot start training: missing data loaders or models")
    print("Make sure the project modules are available and data is loaded")

In [ ]:
# Training function for PyTorch
def train_pytorch_model(model, train_loader, val_loader, num_epochs=EPOCHS):
    """Train PyTorch model with proper training loop"""
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)
    
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}
    best_val_acc = 0.0
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        train_pbar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs} Training')
        for images, labels in train_pbar:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            train_total += labels.size(0)
            train_correct += (predicted == labels).sum().item()
            
            train_pbar.set_postfix({
                'Loss': f'{loss.item():.4f}',
                'Acc': f'{100 * train_correct / train_total:.2f}%'
            })
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            val_pbar = tqdm(val_loader, desc=f'Epoch {epoch+1}/{num_epochs} Validation')
            for images, labels in val_pbar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                val_total += labels.size(0)
                val_correct += (predicted == labels).sum().item()
        
        # Calculate metrics
        epoch_train_loss = train_loss / len(train_loader)
        epoch_train_acc = 100 * train_correct / train_total
        epoch_val_loss = val_loss / len(val_loader)
        epoch_val_acc = 100 * val_correct / val_total
        
        # Update history
        history['train_loss'].append(epoch_train_loss)
        history['train_acc'].append(epoch_train_acc)
        history['val_loss'].append(epoch_val_loss)
        history['val_acc'].append(epoch_val_acc)
        
        # Learning rate scheduling
        scheduler.step(epoch_val_loss)
        
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'Train Loss: {epoch_train_loss:.4f}, Train Acc: {epoch_train_acc:.2f}%')
        print(f'Val Loss: {epoch_val_loss:.4f}, Val Acc: {epoch_val_acc:.2f}%')
        print('-' * 50)
        
        # Save best model
        if epoch_val_acc > best_val_acc:
            best_val_acc = epoch_val_acc
            torch.save(model.state_dict(), 'best_model_colab.pth')
            print(f'New best model saved with validation accuracy: {best_val_acc:.2f}%')
    
    return history

print("✅ PyTorch training function defined")

In [ ]:
# Evaluate models on test set
if 'training_histories' in locals() and training_histories and df is not None:
    test_results = {}
    
    print("🧪 Evaluating models on test set...")
    print("=" * 50)
    
    for name, model in models.items():
        if name not in training_histories:
            continue
            
        model.eval()
        test_loss = 0.0
        test_correct = 0
        test_total = 0
        
        criterion = nn.CrossEntropyLoss()
        
        with torch.no_grad():
            for images, labels in tqdm(test_loader, desc=f'Testing {name}'):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                test_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()
        
        test_accuracy = 100 * test_correct / test_total
        test_loss = test_loss / len(test_loader)
        
        test_results[name] = {
            'accuracy': test_accuracy,
            'loss': test_loss
        }
        
        print(f"{name} Test Results:")
        print(f"  Accuracy: {test_accuracy:.2f}%")
        print(f"  Loss: {test_loss:.4f}")
        print()
    
    # Find best model
    if test_results:
        best_model_name = max(test_results, key=lambda k: test_results[k]['accuracy'])
        best_accuracy = test_results[best_model_name]['accuracy']
        
        print(f"🏆 Best Model: {best_model_name}")
        print(f"🎯 Best Test Accuracy: {best_accuracy:.2f}%")
else:
    print("❌ Cannot evaluate: missing training results or dataset")

In [ ]:
# Train models and store results
if use_project_modules and 'train_loader' in locals():
    training_histories = {}
    
    # Train each model
    for name, model in models.items():
        print(f"\n🚀 Training {name}...")
        print("=" * 50)
        
        try:
            history = train_pytorch_model(model, train_loader, val_loader, num_epochs=EPOCHS)
            training_histories[name] = history
            print(f"✅ {name} training completed successfully!")
        except Exception as e:
            print(f"❌ Error training {name}: {str(e)}")
            continue
        
        print("\n" + "="*50 + "\n")
        
elif not use_project_modules and 'train_df' in locals():
    print("⚠️ Manual training setup needed without project modules")
    print("This would require implementing custom DataLoader for the manual data splits")
    print("For best results, use the complete project environment")
    
else:
    print("❌ Cannot start training: missing data loaders or models")
    print("Make sure the project modules are available and data is loaded")

In [ ]:
# Plot training histories
if 'training_histories' in locals() and training_histories:
    plt.figure(figsize=(15, 5))
    
    # Plot accuracy
    plt.subplot(121)
    for name, history in training_histories.items():
        epochs = range(1, len(history['train_acc']) + 1)
        plt.plot(epochs, history['train_acc'], 'o-', label=f'{name} (train)', alpha=0.8)
        plt.plot(epochs, history['val_acc'], 's-', label=f'{name} (val)', alpha=0.8)
    
    plt.title('Model Accuracy Comparison', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy (%)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    # Plot loss
    plt.subplot(122)
    for name, history in training_histories.items():
        epochs = range(1, len(history['train_loss']) + 1)
        plt.plot(epochs, history['train_loss'], 'o-', label=f'{name} (train)', alpha=0.8)
        plt.plot(epochs, history['val_loss'], 's-', label=f'{name} (val)', alpha=0.8)
    
    plt.title('Model Loss Comparison', fontsize=14, fontweight='bold')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print final results
    print("\n📊 Final Training Results:")
    print("=" * 60)
    for name, history in training_histories.items():
        final_train_acc = history['train_acc'][-1]
        final_val_acc = history['val_acc'][-1]
        print(f"{name}:")
        print(f"  Final Training Accuracy: {final_train_acc:.2f}%")
        print(f"  Final Validation Accuracy: {final_val_acc:.2f}%")
        print()
else:
    print("❌ No training results to display")

In [ ]:
# Evaluate models on test set
if 'training_histories' in locals() and training_histories and df is not None:
    test_results = {}
    
    print("🧪 Evaluating models on test set...")
    print("=" * 50)
    
    for name, model in models.items():
        if name not in training_histories:
            continue
            
        model.eval()
        test_loss = 0.0
        test_correct = 0
        test_total = 0
        
        criterion = nn.CrossEntropyLoss()
        
        with torch.no_grad():
            for images, labels in tqdm(test_loader, desc=f'Testing {name}'):
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                test_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()
        
        test_accuracy = 100 * test_correct / test_total
        test_loss = test_loss / len(test_loader)
        
        test_results[name] = {
            'accuracy': test_accuracy,
            'loss': test_loss
        }
        
        print(f"{name} Test Results:")
        print(f"  Accuracy: {test_accuracy:.2f}%")
        print(f"  Loss: {test_loss:.4f}")
        print()
    
    # Find best model
    if test_results:
        best_model_name = max(test_results, key=lambda k: test_results[k]['accuracy'])
        best_accuracy = test_results[best_model_name]['accuracy']
        
        print(f"🏆 Best Model: {best_model_name}")
        print(f"🎯 Best Test Accuracy: {best_accuracy:.2f}%")
else:
    print("❌ Cannot evaluate: missing training results or dataset")

## 6. Model Speichern und Zusammenfassung

Speichere das beste Modell und erstelle eine Zusammenfassung der Ergebnisse.

In [ ]:
# Save the best performing model
if 'test_results' in locals() and test_results:
    # Create models directory
    os.makedirs('models', exist_ok=True)
    
    # Save the best model
    best_model = models[best_model_name]
    
    # Save complete model for easy loading
    torch.save(best_model, f'models/best_{best_model_name.lower()}_complete.pth')
    
    # Save only state dict (smaller file)
    torch.save(best_model.state_dict(), f'models/best_{best_model_name.lower()}_weights.pth')
    
    print(f"✅ Best model ({best_model_name}) saved successfully!")
    print(f"📁 Complete model: models/best_{best_model_name.lower()}_complete.pth")
    print(f"📁 Model weights: models/best_{best_model_name.lower()}_weights.pth")
    print(f"🎯 Test Accuracy: {best_accuracy:.2f}%")
    
    # Create model summary
    print("\n📋 Training Summary:")
    print("=" * 60)
    print(f"Dataset: HAM10000 (Skin Cancer Detection)")
    print(f"Classes: {NUM_CLASSES} skin lesion types")
    print(f"Training samples: {len(train_df)}")
    print(f"Validation samples: {len(val_df)}")
    print(f"Test samples: {len(test_df)}")
    print(f"Best model: {best_model_name}")
    print(f"Best accuracy: {best_accuracy:.2f}%")
    print(f"Training epochs: {EPOCHS}")
    print(f"Batch size: {BATCH_SIZE}")
    print(f"Device used: {device}")
    
else:
    print("❌ Cannot save model: no test results available")

## 🎉 Zusammenfassung

Dieses Notebook demonstriert eine vollständige PyTorch-basierte Pipeline für die Hautkrebs-Erkennung:

### Was wir erreicht haben:
- **Datenanalyse**: Exploration des HAM10000 Datensatzes
- **Modellvergleich**: EfficientNet, ResNet und Custom CNN
- **Training**: Vollständige PyTorch Trainings-Pipeline
- **Evaluation**: Comprehensive Leistungsbewertung
- **Deployment**: Modell für Produktivumgebung bereit

### Medizinischer Kontext:
- 7 verschiedene Hautläsionstypen klassifiziert
- Unterstützt medizinisches Fachpersonal bei der Voruntersuchung
- Wichtig: Nur für Bildungs- und Forschungszwecke

### Nächste Schritte:
1. Modell in Streamlit App integrieren
2. Weitere Datenaugmentation testen
3. Hyperparameter-Optimierung
4. Ensemble-Methoden evaluieren

**⚠️ Medizinischer Hinweis**: Dieses Tool dient nur zu Bildungs- und Forschungszwecken. Konsultieren Sie immer qualifizierte medizinische Fachkräfte für medizinische Diagnosen.